In [1]:
import json
from datasets import load_dataset, Dataset
# Specify the path to the JSON file
file_path = 'datasets/squad-sr-lat.json'

# Open the JSON file and load its contents
with open(file_path, 'r', encoding='utf8') as file:
    data = json.load(file)

# Now, 'data' contains the parsed JSON data as a Python dictionary
print(data.keys())


dict_keys(['version', 'data'])


In [3]:
# sentence1 = []
# sentence2 = []
# data_for_eval = {
#     'id': [],
#     'title': [],
#     'context': [],
#     'question': [],
#     'answers': []
# }
# i = 1
# for article in data['data']:
#     title = article['title']
#     for paragraph in article['paragraphs']:
#         context = paragraph['context']
#         for qa_pair in paragraph['qas']:
#             question = qa_pair['question']
#             data_for_answer = {
#                 "text": [],
#                 'answer_start': []
#             }
#             for answer in qa_pair['answers']:
#                 short_answer = answer['text']
#                 answer_start = answer['answer_start']
#                 sentence1.append(question)
#                 sentence2.append(short_answer)
#                 data_for_answer['text'].append(short_answer)
#                 data_for_answer['answer_start'].append(answer_start)
#             data_for_eval['id'].append(str(i))
#             data_for_eval['title'].append(title)
#             data_for_eval['context'].append(context)
#             data_for_eval['question'].append(question)
#             data_for_eval['answers'].append(data_for_answer)
#             i += 1
#                 # print(answer)
# dataset_for_eval = Dataset.from_dict(data_for_eval)

In [4]:
# from evaluate import evaluator
# task_evaluator = evaluator("question-answering")
# # data = load_dataset("squad_v2", split="validation[:2]")
# results = task_evaluator.compute(
#     model_or_pipeline="classla/bcms-bertic",
#     data=dataset_for_eval,
#     metric="squad_v2",
#     squad_v2_format=True,
# )

In [5]:
# results

loading the data to wanted format

In [6]:
i = 1
titles = []
contexts = []
queryes = []
for article in data['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        currect_queryes = []
        for qa_pair in paragraph['qas']:
            question = qa_pair['question']
            
            currect_queryes.append(question)
            i += 1
        titles.append(title)
        contexts.append(context)
        queryes.append(currect_queryes)
                # print(answer)

In [7]:
import pandas as pd
# from collections import defaultdict
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queryes})

dataset_for_eval_df.head()
dataset_for_eval_df['queries'][0]

['Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?',
 'Šta je ispred Glavne zgrade Notr Dame?',
 'Bazilika Svetog srca u Notr Dami je pored koje strukture?',
 'Šta je Grota u Notr Dami?',
 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?']

In [8]:
import pickle
import pyarrow as pa
import pyarrow.parquet as pq

dataset_for_eval_df.to_csv('datasets/dataset_processed.csv', index=False)
dataset_for_eval_df.to_pickle('dataset_for_eval.pkl')
table = pa.Table.from_pandas(dataset_for_eval_df)

pq.write_table(table, 'datasets/dataset_processed.parquet')


In [9]:
dataset_for_eval_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18896 entries, 0 to 18895
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    18896 non-null  object
 1   context  18896 non-null  object
 2   queries  18896 non-null  object
dtypes: object(3)
memory usage: 443.0+ KB


making the pipieline for all models

In [10]:
# Create model embeddings
# def generate_embeddings(text):
# Initialize the question-answering pipeline
# Measuring query-context similarity
# def compute_similarity(query, context, top_k):
# Generate predictions

In [11]:
# import pandas as pd
# import torch
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# # Load the model and tokenizer
# model_name = "classla/bcms-bertic"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# # Load the dataset
# df = pd.read_csv('datasets/dataset_processed.csv')

# # Function to get the answer from the context based on the predicted start and end positions
# def get_answer_from_positions(context, start_pos, end_pos):
#     return context[start_pos:end_pos+1]

# # Tokenize the context and query, and get predictions
# def get_predictions(context, query):
#     inputs = tokenizer.encode_plus(query, context, return_tensors='pt')
#     input_ids = inputs['input_ids']
#     token_type_ids = inputs['token_type_ids']
#     attention_mask = inputs['attention_mask']
    
#     with torch.no_grad():
#         outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
#         start_scores, end_scores = outputs.start_logits, outputs.end_logits
    
#     start_pos = torch.argmax(start_scores)
#     end_pos = torch.argmax(end_scores)
    
#     answer_tokens = input_ids[0][start_pos:end_pos+1]
#     answer = tokenizer.decode(answer_tokens)
#     return answer

# # Evaluate the model
# predictions = []
# for idx, row in df.iterrows():
#     context = row['context']
#     queries = row['queries']
#     for query in queries:
#         prediction = get_predictions(context, query)
#         predictions.append(prediction)

# df['predicted_answer'] = predictions

# # (Optional) Save the predictions to a new CSV file
# df.to_csv('dataset_with_predictions.csv', index=False)


In [12]:
import pandas as pd
import torch
import numpy as np
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel
from collections import defaultdict
import pytrec_eval
import json
from sklearn.metrics.pairwise import cosine_similarity



1. ucitati model i tokenizer
2. ucitati df
3. prolazimo kroz sve recorde u df i za svaki record
    1. ubacimo listu query u uciran model i dobije listu embedinga
    2. ubacimo context u model i dobijemo encodirano
    3. merimo slicnost izmedju embedinga i matrice vecContexta@matriceVektoraQuerya
    pored cosinusne se cuva i index contexta
    nama treba i za query da se cuva koji je njegog paragraf
    4. sortiramo sve umnoske, iyvucemo najslicniji

In [15]:
loaded_table = pq.read_table("datasets/dataset_processed.parquet")
df = loaded_table.to_pandas()
df.head()

,title,context,queries
0,University_of_Notre_Dame,"Arhitektonski, škola ima katolički karakter. N...","[Kome se u 1858. godini u Lurdu, Francuska, na..."
1,University_of_Notre_Dame,"Kao i na većini drugih univerziteta, studenti ...",[Kada je počelo izdavanje Scholastičkog časopi...
2,University_of_Notre_Dame,Univerzitet je glavna sedište Kongregacije Sve...,"[Gde je sedište Kongregacije Svetog Krsta?, Št..."
3,University_of_Notre_Dame,Koledž za inženjerstvo osnovan je 1920. godine...,[Koliko diploma na nivo BS se nudi u Koledžu z...
4,University_of_Notre_Dame,Svi studenti Notr Dame-a su deo jednog od pet ...,[Koje su subjekti koje pružaju pomoć u upravlj...


In [16]:
#mead puling
def get_sentence_embedding(sentence, model, tokenizer): 
    """""
    Pitanja:
    - Da li koristiti mean pooling ili CSL pooling?
    - U SQuAD datasetu su queries string i treba ih parsirati u listu
    """""
    encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
    attention_mask = encoded_input['attention_mask']   # to indicate which tokens are valid and which are padding
    
    # Get the model output (without the specific classification head)
    with torch.no_grad():
        output = model(**encoded_input)

    token_embeddings = output.last_hidden_state # poslednji nivo
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

    # mean pooling operation, considering the BERT input_mask and padding
    sentence_embedding = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    return sentence_embedding.flatten().tolist()
    # return sentence_embedding


def cos_sim(context_embed, query_embed):
    return context_embed @ query_embed.T


def evaluate(model_name, dataset_path):
    # df = pd.read_csv(dataset_path)
    # df = pd.read_pickle('dataset_for_eval.pkl') # picke jer je bio problem ucitati df preko pandas. Queries je string i treba parsirati u listu
    # with open('dataset_for_eval.pkl', 'rb') as file:
    # # Load the pickled data
    #     byte_data = file.read()

    # # Decode the byte data as UTF-8 if necessary
    # # Normally you would not do this step, as pickle handles it
    # utf8_data = byte_data.decode('utf-8')

    # # Now load it using pickle, if it was necessary to decode (rare)
    # df = pickle.loads(utf8_data.encode('utf-8'))
    # with open('income.pickle', 'rb') as f:  # with statement avoids file leak
    #     # Match load with file object, and provide encoding for Py2 str
    #     df = pickle.load(f, encoding='utf-8')

    loaded_table = pq.read_table("datasets/dataset_processed.parquet")
    df = loaded_table.to_pandas()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    ground_truth = {}# ovo je nase
    # qrel = {
    #     "tekst_query1":
    #     {"1_id_paragrafa": 0,
    #     "2_id_paragrafa": 1,
    #     "3_id_paragrafa": 0},
    #     "tekst_query2":
    #     {"1_id_paragrafa": 1,
    #     "2_id_paragrafa": 0,
    #     "3_id_paragrafa": 0},
    # }
    retrieved_scores = {}  # ovo su cosinusne

    #we embed the contexts
    context_embeddings = {}
    for idx, row in df.iterrows():
        # if idx >= 2:  # Break the loop after two iterations
        #     break
        context = row['context']
        embedded_context = get_sentence_embedding(context, model, tokenizer)
        context_embeddings[str(idx)] = embedded_context
        # print(idx)
    
    # we fill the dict used for metrics
    for idx, row in df.iterrows():
        # if idx >= 2:  # Break the loop after two iterations
        #     break
        queries = row['queries'] # [1:-1].split("', ' ") zato sto je sacivan kao string pa mora u listu da se pretvara
        # print(queries)
        # print(type(queries))
        for query in queries:
            query = query.strip()
            # print(query)
            ground_truth[query] = {}
            retrieved_scores[query] = {}
            embedded_query = get_sentence_embedding(query, model, tokenizer)
            for current_context_id, current_context_embedding in context_embeddings.items():
                ground_truth[query][current_context_id] = 0
                # score = cos_sim(current_context_embedding, embedded_query)
                # print(embedded_query)
                score = cosine_similarity(np.array(current_context_embedding).reshape(1, -1), np.array(embedded_query).reshape(1, -1))
                # print(score.item())
                retrieved_scores[query][current_context_id] = score.item()
            ground_truth[query][str(idx)] = 1 # samo ovaj kome pripada je 1 ostali su nula, necemo da punimo jer je default dict
            
    
    evaluator = pytrec_eval.RelevanceEvaluator(
    ground_truth, {'map', 'ndcg'}
    ) 
    # print(context_embeddings)
    print(json.dumps(evaluator.evaluate(retrieved_scores), indent=1))
    # evaluator.evaluate(run)
    return retrieved_scores
     




run = evaluate("classla/bcms-bertic", 'datasets/dataset_processed.csv')

/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
print(run)


{'Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?': {'0': 0.9702974328351258, '1': 0.9417458116887851}, 'Šta je ispred Glavne zgrade Notr Dame?': {'0': 0.9398622207027303, '1': 0.9148289297375717}, 'Bazilika Svetog srca u Notr Dami je pored koje strukture?': {'0': 0.9571723517991639, '1': 0.9233963285326181}, 'Šta je Grota u Notr Dami?': {'0': 0.9302698055098331, '1': 0.8982230382425597}, 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?': {'0': 0.97306117336876, '1': 0.9459882890289258}, 'Kada je počelo izdavanje Scholastičkog časopisa Notr dame?': {'0': 0.9430615139247354, '1': 0.909179445095098}, 'Koliko često se "Notr Dams džongler" objavljuje?': {'0': 0.9401447997890098, '1': 0.9091041988650693}, 'Kako se zove dnevni studentski list u Notr Damu?': {'0': 0.9681517831548883, '1': 0.9366752558501559}, 'Koliko studentskih novinara se nalazi na Notr Dame?': {'0': 0.9355253300773242, '1': 0.9058199066693698}, 'U kojoj godini je studentski list "Sporočni razum" p

In [ ]:
# sentence transformer

# batching u sentence transformerima i trnasformerima, 

# embedowan query i array svih conteksta u 1 calling, sortiraju se, top k, dobiju se indexi

#clod